In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("D:\CoachBot\SFTuned")
tokenizer=AutoTokenizer.from_pretrained("D:\CoachBot\SFTuned", use_fast=False)

In [2]:
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig
#ds = load_dataset("entfane/psychotherapy-dpo")
ds = load_dataset("hungtuchen/MentalLlama-dpo-pairs")

In [3]:
ds = ds.rename_column("Context", "prompt")
print(ds["train"].column_names)  # 应该显示 ['prompt','chosen','rejected']

['prompt', 'chosen', 'rejected']


In [4]:
'''ds = ds.remove_columns("illness")
ds = ds.rename_column("input", "prompt")
print(ds["train"].column_names)  # 应该显示 ['prompt','chosen','rejected']'''

'ds = ds.remove_columns("illness")\nds = ds.rename_column("input", "prompt")\nprint(ds["train"].column_names)  # 应该显示 [\'prompt\',\'chosen\',\'rejected\']'

In [5]:
len(ds["train"])

352

In [6]:
from peft import get_peft_config, get_peft_model, LoraConfig

In [7]:
lora_config = LoraConfig(
    r=8,                      
    target_modules=["q_proj","v_proj", "k_proj", "o_proj", "dense_h_to_4h"],  
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, lora_config)

n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("可训练参数总数：", n_trainable)

可训练参数总数： 2293760


In [8]:
training_args = DPOConfig(
    output_dir="./checkpoints",            
    per_device_train_batch_size=2,
    gradient_accumulation_steps=6,
    learning_rate=1e-5,
    num_train_epochs=3,
    logging_steps=5,
    save_strategy="steps",
    save_steps=5,
    save_total_limit=2, 
    gradient_checkpointing=True

)
model.gradient_checkpointing_enable()
trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],   
    tokenizer=tokenizer,
)

C:\Users\AriZu\AppData\Local\Temp\ipykernel_28700\2156461788.py:15: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  trainer = DPOTrainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
trainer.train()

model.save_pretrained("./DPO_1")
tokenizer.save_pretrained("./DPO_1")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
5,0.691000
10,0.678100
15,0.682800
20,0.675800
25,0.669200
30,0.596300
35,0.570400
40,0.551200
45,0.538100
50,0.546100


c:\Users\AriZu\.conda\envs\train\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:\CoachBot\SFTuned - will assume that the vocabulary was not modified.
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:\CoachBot\SFTuned - will assume that the vocabulary was not modified.
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:\CoachBot\SFTuned - will assume that the vocabulary was not modified.
  warni

('./DPO_1\\tokenizer_config.json',
 './DPO_1\\special_tokens_map.json',
 './DPO_1\\vocab.json',
 './DPO_1\\merges.txt',
 './DPO_1\\added_tokens.json')